# Optimizing with Psi4

In the previous notebook we created a representation of a water molecule based on a text string that describes the atoms and their geometry. We will now take this geometry as a starting position and optimize the structure to obtains the geomtery with the lowest energy.

As always, we begin with loading the packages needed. Run the code below to do this.

You may find the need for more information than what I present here. I suggest consulting the **[*Psi4* website](https://psicode.org)** and the documentation and Q&A forums within. Much of this discussion is based on a tutorial of *Psi4* found **[here](https://github.com/Jammyzx1/psi4-nb/blob/main/psi4_intoductory_methods.ipynb)**.

In [1]:
# use psi4conda environment
import psi4
import os
import numpy as np
print('done')  # It can take time to load in new libraries. With this print
               # command I know when that is finished and I can proceed.

done


## First Things Again

At the start we will repeat the build of the ```Molecule``` object that describes water. Run the code below.


In [21]:
output_file = "H2O_optimize.out"
psi4.set_memory("1GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

# The Z-matrix as a text string
data = """
       0 1
       O1
       H2  1  length
       H3  1  length  2  angle
       length = 1.0
       angle = 110
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string


## Your First Optimization

We now have a ```Molecule``` object that describes an initial geometry of water. If we are close to the true structure of water, we will have a clear downhill "slope" in energy toward the lowest energy structure.  

Optimization works by calculating the derivative of the energy function for the nuclei and electrons in the current geometry using the math of quantum chemistry. This derivative defines and energy gradient for each atom. The program then moves each atom along that gradient. We now have a new geometry that is hopefully lower in energy. Now we repeat the whole process again and obtain another geometry that is hopfully lower in energy again. Eventually the gradient (and the required movements of atoms) is so small that we start going back and forth. We have reached the bottom of the energy well and are now vibrating. We should stop now. When the energy gradient from a calculation reaches a set tolerance value the program will halt.

How the energy and the gradient are calculated will not be discussed here. We will revisit this subject briefly when we discuss energy and wavefunctions in the next notebook. For now, be aware that the choice of method (Hartee-Fock, Self-Consistent Field Theory, dentity Functional Theory, etc.) and the choice of mathematical model for atomic orbitals (the "basis set") will decide the accuracy and the cost of your calculations.

### Setting Up the Calculation

*Psi4* can calculate the energy and atomic gradients for a geometry. We could then apply these gradients and calculate a new set of coordinats for the geometry and repeat. As short *Python* script would do this using the components of the *Psi4* package. However, there is an easier way.

*Psi4* has a set of "driver" programs that will perform multistep calculations for us and report the results along the way in the output file. The driver that we will use is the ```psi4.optimize()``` function. As stated above, the method and basis set must be chose. there are several other imprtant variables that can be set as we prepare to optimize.

We will use the ```psi4.set_options()``` function to set a group of options for our optimization. We need to pass a dictionary of the option labels and corresponding values into the function. Consider the code below.

You can get a list of all variables in *Psi4* by executing the command ```psi4.core.print_global_options()```. You can reset the options to defaults with ```psi4.core.clean_options()```.

In [22]:
psi4.set_options({
        "BASIS": "cc-pvdz",           # default => None - Basis set must be specified
        "SAVE_OPTIMIZATION": True,    # default => False
        "MAXITER": 100,               # default => 50
        "GEOM_MAXITER": 100,          # default => 50
        "FULL_HESS_EVERY": 5,         # default => -1
        "PRINT": 2,                   # default => 1
        "GUESS": "sad",               # default => auto
        "REFERENCE": "rhf",           # default => rhf
        "SCF_TYPE": "direct",         # default => pk
        "INTS_TOLERANCE": 1E-8        # default => 1e-12. A value of 1e-8 is recommended when SCF_TYPE set to "direct"
    })

### Performing an Energy Calculation

We can then obtain the energy for a given geometry by passing the ```Molecule``` object into the ```psi4.energy()``` function. The first argument is the method (hf, scf, mp2, etc.) It is required. The second argument is the optional declaration of the ```Molecule``` object to use. By default the most recently created ```Molecule``` object will be used.

The basis set needs to be set in the ```psi4.set_options()``` function as described above or it can be input directly in the ```psi4.energy()``` function in combination with the method. For example ```psi4.set_options("hf/cc-pvdz")``` would suffice. The cc-pvdz basis set is a less costly basis set (and less accurate, but good enough for now.)

The energy is reported in Hartree atomic units. One hartree is 2625.5 kJ/mole and can be more precisely accessed as the constant ```psi4.constants.hartree2kJmol```.

In [13]:
energy = psi4.energy("hf", molecule = mol)
print("The energy for the initial configuration is {:.7f} Hartree\n".format(energy))

The energy for the initial configuration is -76.0201398 Hartree



### Performing the Optimization Calculation

We can then perform the optimization on a ```Molecule``` object. Observe above that I had named our ```Molecule``` object as "mol". The input for ```psi4.optimize()``` is the method and the molecule as described above. there are options for these functions. Just use the ```help()``` function to extract the documentation of a function. For example ```help(psi4.optimize)``` will provide information about this function. Among the options available is the ```return_history``` flag. Setting it to true will return a dictionary of energies and coordinates for each step of the optimization. You can get the keys for a dictionary with the ```.keys()``` method built into all dictionary objects. Run the command ```hist.keys()``` to get the keys to ```hist```.

In [14]:
opt_energy, hist = psi4.optimize("hf", molecule = mol, return_history=True)
print(f"The optimized energy for this configuration is {opt_energy:.7f} Hartree\n")

e_diff = (opt_energy - energy) * psi4.constants.hartree2kJmol
print(f"The difference in energy between in initial and optimized geomtery is {e_diff:.2f} kJ/mole\n")

energy_list = np.array(hist["energy"])
print("The energies as the optimization proceeds is...")
print(energy_list,"Hartrees")
print("The relative energies at each step of optimization are...")
energy_list = (energy_list - energy_list[0]) * psi4.constants.hartree2kJmol
print(energy_list,"kJ/mole")


Optimizer: Optimization complete!
The optimized energy for this configuration is -76.0270535 Hartree

The difference in energy between in initial and optimized geomtery is -18.15 kJ/mole

The energies as the optimization proceeds is...
[-76.02013978 -76.02675888 -76.02704384 -76.02705316 -76.02705350] Hartrees
The relative energies at each step of optimization are...
[  0.00000000 -17.37846177 -18.12662595 -18.15108714 -18.15197668] kJ/mole


### A Frequency calculation

We can obtain frequency information in the optimized structure using the ```psi4.frequency()``` function. It also has options like the calculators above. In this case we are using the ```return_wfn``` flag to ask for an object containg the data for all the wavefunctions to be returns. The wavefunction object can be used to calculate electronic properties such as charges and electron densities.

The ```Molecule``` object that had been passed to the ```psi4.optimize()``` function will have been altered by that function. It will contain the optimized structure.

In [10]:
vib_energy, wfn = psi4.frequency("hf", molecule = mol, return_wfn=True)
print(f"The  energy for this configuration is {vib_energy:.7f}")

The  energy for this configuration is -76.0270535


### Writing Wavefunction Info

Many external programs can display the molecular orbitals. The *fchk* file format is used by many of these programs. The ```psi4.driver.fchk()``` function will take the wavefunction object and output it in *fchk* format to the given filename.



In [7]:
psi4.driver.fchk(wfn, "H2O_out.fchk")

## Examine the Output File

The output file for the commands above is ```H2O_optimize.out```. Open it and you will see that it is very large. Go right to the end and you will find all the answers you are looking for.  

In the last lines of the file you will find the geometry of the atoms expressed as cartesian coordinates and as a Z-matrix (You will get a final geomtry reported in the format that was used to create the ```Molecule``` object. You will find the energy (in atomic units aka Hartrees) and also lists of bond lengths, bond angles and tortions (none in this case).

This set of information appears for each structure found along the path from start to finish. In my own calculation as described in this notebook the optimization took five steps. We now have an optimized structure and the energy for that structure.

### Deep Output Diving

The output file has a wealth of information. Depending on settings it will contain wavefunction and vibrational data also. It might also contain a detailed thermochemical analysis. Every time we add a calculation, more information is appended to the file. After we add several more operations it might be difficult to quickly find the results of the final optimization step. We can easily solve this problem by using simple search functions in a text editor or by using the powerful text manipulation tools of *Unix* such as *grep*.

Rather than leave this notebook to use external programs, we will stay here and use a tool that will parse the output file and pull out lots of useful data. this tool is *cclib*.

### Introduction to cclib

*cclib* will parse the output file of many different quatum chemistry packages. Unless there is a serious problem with your output file it will automaticall recognize the format as *Psi4* and act accordingly. Consider the code below. There is even more data contained in the text file itslef but *cclib* is a convenient way to access much of it from within *Python* (and it will be even more useful once these bugs get fixed - or I could fix them myself and contribute the code to the project).

*cclib* finds information from the text output file and populates variables and data objects within the *cclib* object. Just some of the data scraped from the file is presented in the code below. Use ```help(data)``` and ```dir(data)``` to get more information on the ```data``` object created in the code below. 

**cclib** is a great way to access the output file (when it works) but you can also access all that info directly using a text editor. In our examples we will rarely need more than the info available in the wavefunction, hessian and history objects that can be returned from the energy, optimize and frequency functions of *Psi4*.

**NOTE:** *cclib* is currently buggy and the code below will not work on optimization output files. I will leave this section here so that I can update it in future. I guess you will have to turn to *grep*.

**NOTE:*** I editted the ```psi4parser.py``` file in the *cclib* package and commented-out the problem code (two lines). I also need to edit the output file to remove the version line at the start of the file. With this workaround I got *cclib* to parse my outoput file.

In [9]:
import cclib

# The line in the output file that gives us problems has a series of "z" characters
# for the vesrion string. cclib chokes on this non-standard versioning
# We can run a grep command in terminal as shown below that will remove the
# line containing "zzzz" and write all the other lines into a new file.
# We will then use the edited cclib package (fixed another bug) to parse the output.
!grep -v zzzz H2O_optimize.out > H2O_optimize2.out

output_file = "H2O_optimize2.out"

parser = cclib.io.ccopen(output_file)
data = parser.parse()

print(f"There are {data.natom} atoms and {data.nmo} MOs")
print("The vibrational frequencies are...")
print(data.vibfreqs)
print("The vibrational force constants are...")
print(data.vibfconsts)
print("The vibrational symmetry are...")
print(data.vibsyms)


There are 3 atoms and 24 MOs
The vibrational frequencies are...
[ 1775.96690000  4112.68610000  4211.04440000]
The vibrational force constants are...
[  2.01020000  10.42460000  11.30440000]
The vibrational symmetry are...
['A1', 'A1', 'B2']


## Conclusion

We have taken a molecule and determined it energy before and after optimization using the ```psi4.energy()``` function and the psi4.optimize() functions. We have seen how to access the energies and geometries of every step in the optimization using the ```return_history``` flag and how to get the wavefunction data using the ```return_wfn``` flag. We have calculated vibrational data of the optimized structure using the ```psi4.fequency()``` function.

We now have an output file that contains the results of all of the above calculations and have created an output file in the *Guassian .fchk* format. These files can be used by external visualization program to analyzed the data in more detail.

*ChimeraX* with the *SeqCrow* plugin can read the output file and visualize the molecule, its vibrational vectors and orbitals. *Avogadro* can read the *fchk* file and present the orbitals. If you want to go hard-core, *MultiWFN* can analyze the output data and present detailed information about the electronic structure of a given geometry. There are many more tools to try as well.

We can now obtain the minimized structure for a molecule (in gas phase) and then examine it and comment on the reasons for the given steric arrangement. But what if we want to explore a structure other than the lowest energy case? for example, we may want to examine ethan in the *anti* configuration, or cyclohexane in its "boat but not twist-boat" form. We may also want to catch a reaction in the act and model the transition state. To do this we must now consider how to constrain elements of the geometry. That is our next topic.